In [20]:
import time
import os
import numpy as np 
from pathlib import Path
import argparse
import ffsim
import scipy
import ffsim
import pyscf
from collections import defaultdict
from sqd_lattice.pre_process import make_pyscf_slater_det,make_two_body_tensor
from sqd_lattice.util import load_pickle,load_yaml,save_pickle,load_json,save_json
from ffsim import FermionOperator,cre_a,cre_b,des_a,des_b,linear_operator,slater_determinant,hartree_fock_state

In [21]:
material = 'platinum_no_U'
ne = "30"

folder_path = f"../runs/{material}/{ne}e"

# # Loading data
base_config = load_yaml(f"../runs/{material}","base_config.yaml")
data = load_pickle(f"../runs/{material}","data_dict.pkl")
results_pre_process = load_pickle(folder_path,"results_pre_process.pkl")
num_orbitals = data["hopping_matrix"].shape[0]
hopping_matrix = data["hopping_matrix"]

Hamiltonian in the atomic basis, using the right slater determinant

In [3]:
ham = ffsim.MolecularHamiltonian(one_body_tensor=data["hopping_matrix"],two_body_tensor=data["two_body_tensor"])

In [4]:
lin_op = linear_operator(ham,num_orbitals,results_pre_process["ne_ab"])

hopping_diag = np.diag(hopping_matrix)

occupied_orbitals_a = np.argsort(hopping_diag)[:results_pre_process["ne_ab"][0]]
occupied_orbitals_b = np.argsort(hopping_diag)[:results_pre_process["ne_ab"][1]]
occupied_orbitals = [occupied_orbitals_a,occupied_orbitals_b]
state = slater_determinant(num_orbitals,occupied_orbitals=occupied_orbitals)

# state = hartree_fock_state(num_orbitals,results_pre_process["ne_ab"])

energy = np.real(np.vdot(state, lin_op @ state))
energy

np.float64(-628.4096732920424)

In [5]:
sum(2*np.sort(hopping_diag)[:15])

np.float64(-628.4096732920425)

In [6]:
results_pre_process["hf_energy"]

np.float64(-662.3966533213743)

In [7]:
ham = ffsim.MolecularHamiltonian(one_body_tensor=data["hopping_matrix"],two_body_tensor=data["two_body_tensor"])
ham = ham.rotated(results_pre_process["mo_coeff"].T)

In [8]:
lin_op = linear_operator(ham,num_orbitals,results_pre_process["ne_ab"])
state = hartree_fock_state(num_orbitals,results_pre_process["ne_ab"])

energy = np.real(np.vdot(state, lin_op @ state))
energy

np.float64(-662.3966533213744)

In [9]:
results_pre_process["hf_energy"]

np.float64(-662.3966533213743)

In [10]:
eigval,eigvec = scipy.linalg.eig(data["hopping_matrix"])
hop_diag = np.diag(eigval)

In [11]:
hop_diag

array([[-99.3498924 +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j],
       [  0.        +0.j, -53.14251935+0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j],
       [  0.        +0.j,   0.        +0.j, -53.53998592+0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
      

In [12]:
ham = ffsim.MolecularHamiltonian(one_body_tensor=hop_diag,two_body_tensor=data["two_body_tensor"])

In [13]:
lin_op = linear_operator(ham,num_orbitals,results_pre_process["ne_ab"])

occupied_orbitals_a = np.argsort(eigval)[:results_pre_process["ne_ab"][0]]
occupied_orbitals_b = np.argsort(eigval)[:results_pre_process["ne_ab"][0]]
occupied_orbitals = [occupied_orbitals_a,occupied_orbitals_b]
state = slater_determinant(num_orbitals,occupied_orbitals=occupied_orbitals)

# state = hartree_fock_state(num_orbitals,results_pre_process["ne_ab"])

energy = np.real(np.vdot(state, lin_op @ state))
energy

np.float64(-662.3966533213742)

In [15]:
results_pre_process["hf_energy"]

np.float64(-662.3966533213743)

In [16]:
ham = ffsim.MolecularHamiltonian(one_body_tensor=hopping_matrix,two_body_tensor=data["two_body_tensor"])
ham = ham.rotated(eigvec.T)

In [17]:
lin_op = linear_operator(ham,num_orbitals,results_pre_process["ne_ab"])

occupied_orbitals_a = np.argsort(eigval)[:results_pre_process["ne_ab"][0]]
occupied_orbitals_b = np.argsort(eigval)[:results_pre_process["ne_ab"][0]]
occupied_orbitals = [occupied_orbitals_a,occupied_orbitals_b]
state = slater_determinant(num_orbitals,occupied_orbitals=occupied_orbitals)

# state = hartree_fock_state(num_orbitals,results_pre_process["ne_ab"])

energy = np.real(np.vdot(state, lin_op @ state))
energy

np.float64(-662.3966533213744)

In [19]:
sum(2*np.sort(eigval)[:15])

np.complex128(-662.396653321374+0j)